In [1]:
import pandas_gbq
import requests
import datetime
import json
import pandas
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gspread
import pandas as pd

In [2]:
key_path = '/Users/aleksandra/Desktop/PRO/foodru-a879e-fa90e9d9e0d2.json'
gbq_credential = service_account.Credentials.from_service_account_file(key_path,)

In [3]:
pd.set_option('display.max_columns', None)

In [ ]:
# date = 20230216

In [4]:
q = '''-- пользователи с базовыми метриками
WITH users AS (
  SELECT  
    platform
    ,(SELECT user_properties.value.string_value FROM UNNEST(user_properties)user_properties WHERE user_properties.key = 'mindbox_id') mindbox_id
    ,COUNT(DISTINCT event_date) cnt_days
    ,COUNT(CASE WHEN event_name = 'screen_view' THEN event_timestamp END) cnt_screen_view  
    ,COUNT(CASE WHEN event_name = 'fd_RecipePage_view' THEN event_timestamp END) cnt_recipe_view 
  FROM `foodru-a879e.analytics_269143280.events_*`
  WHERE 1=1
      AND NOT app_info.id LIKE '%dev%'
      AND _TABLE_SUFFIX >= '20230216'
  GROUP BY 1,2
)

 -- рассчёт объёма сегментов
, cnt_segments AS (
  SELECT
    platform
    ,COUNT(DISTINCT CASE WHEN cnt_days >=2 AND cnt_recipe_view >=5 THEN mindbox_id END) cnt_1
    ,COUNT(DISTINCT CASE WHEN cnt_days >=3 AND cnt_recipe_view >=10 THEN mindbox_id END) cnt_2
    ,COUNT(DISTINCT CASE WHEN cnt_days >=5 AND cnt_recipe_view >=15 THEN mindbox_id END) cnt_3
    ,COUNT(DISTINCT CASE WHEN cnt_days >=7 AND cnt_recipe_view >=20 THEN mindbox_id END) cnt_4
    ,COUNT(DISTINCT CASE WHEN cnt_days >=10 AND cnt_recipe_view >=30 THEN mindbox_id END) cnt_5
  FROM users
  WHERE 1=1
    AND mindbox_id IS NOT NULL
  GROUP BY 1
  ORDER BY 1
)

 -- список пользователей
, all_users_segments AS (

  SELECT DISTINCT 
    '1. days >=2 & recipe_view >=5' segment
    ,platform
    ,mindbox_id
  FROM users
  WHERE 1=1
    AND mindbox_id IS NOT NULL
    AND cnt_days >=2 AND cnt_recipe_view >=5

UNION ALL 

  SELECT DISTINCT 
    '2. days >=3 & recipe_view >=10' segment
    ,platform
    ,mindbox_id
  FROM users
  WHERE 1=1
    AND mindbox_id IS NOT NULL
    AND cnt_days >=3 AND cnt_recipe_view >=10

UNION ALL 

  SELECT DISTINCT 
    '3. days >=5 & recipe_view >=15' segment
    ,platform
    ,mindbox_id
  FROM users
  WHERE 1=1
    AND mindbox_id IS NOT NULL
    AND cnt_days >=5 AND cnt_recipe_view >=15

UNION ALL 

  SELECT DISTINCT 
    '4. days >=7 & recipe_view >=20' segment
    ,platform
    ,mindbox_id
  FROM users
  WHERE 1=1
    AND mindbox_id IS NOT NULL
    AND cnt_days >=7 AND cnt_recipe_view >=20

UNION ALL 

  SELECT DISTINCT 
    '5. days >=10 & recipe_view >=30' segment
    ,platform
    ,mindbox_id
  FROM users
  WHERE 1=1
    AND mindbox_id IS NOT NULL
    AND cnt_days >=10 AND cnt_recipe_view >=30
)

SELECT * FROM all_users_segments ORDER BY 1,2

'''

In [5]:
mods = pandas_gbq.read_gbq(q, project_id='foodru-a879e', credentials=gbq_credential)
mods

/Users/aleksandra/Desktop/PRO/env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2007: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,segment,platform,mindbox_id
0,1. days >=2 & recipe_view >=5,ANDROID,1671525
1,1. days >=2 & recipe_view >=5,ANDROID,1702765
2,1. days >=2 & recipe_view >=5,ANDROID,1849953
3,1. days >=2 & recipe_view >=5,ANDROID,1962300
4,1. days >=2 & recipe_view >=5,ANDROID,1956534
...,...,...,...
135000,5. days >=10 & recipe_view >=30,IOS,1733534
135001,5. days >=10 & recipe_view >=30,IOS,1703283
135002,5. days >=10 & recipe_view >=30,IOS,857666
135003,5. days >=10 & recipe_view >=30,IOS,1214934


In [6]:
mods['segment'].unique()

array(['1. days >=2 & recipe_view >=5', '2. days >=3 & recipe_view >=10',
       '3. days >=5 & recipe_view >=15', '4. days >=7 & recipe_view >=20',
       '5. days >=10 & recipe_view >=30'], dtype=object)

In [7]:
s1_ios = mods[(mods['segment'] == '1. days >=2 & recipe_view >=5') & (mods['platform'] == 'IOS')]
s1_android = mods[(mods['segment'] == '1. days >=2 & recipe_view >=5') & (mods['platform'] == 'ANDROID')]

s2_ios = mods[(mods['segment'] == '2. days >=3 & recipe_view >=10') & (mods['platform'] == 'IOS')]
s2_android = mods[(mods['segment'] == '2. days >=3 & recipe_view >=10') & (mods['platform'] == 'ANDROID')]

s3_ios = mods[(mods['segment'] == '3. days >=5 & recipe_view >=15') & (mods['platform'] == 'IOS')]
s3_android = mods[(mods['segment'] == '3. days >=5 & recipe_view >=15') & (mods['platform'] == 'ANDROID')]

s4_ios = mods[(mods['segment'] == '4. days >=7 & recipe_view >=20') & (mods['platform'] == 'IOS')]
s4_android = mods[(mods['segment'] == '4. days >=7 & recipe_view >=20') & (mods['platform'] == 'ANDROID')]

s5_ios = mods[(mods['segment'] == '5. days >=10 & recipe_view >=30') & (mods['platform'] == 'IOS')]
s5_android = mods[(mods['segment'] == '5. days >=10 & recipe_view >=30') & (mods['platform'] == 'ANDROID')]

In [12]:

with pd.ExcelWriter("data.xlsx") as writer:
    mods.to_excel(writer, sheet_name="raw")
    s1_ios.to_excel(writer, sheet_name="s1_ios")
    s1_android.to_excel(writer, sheet_name="s1_android")
    s2_ios.to_excel(writer, sheet_name="s2_ios")
    s2_android.to_excel(writer, sheet_name="s2_android")
    s3_ios.to_excel(writer, sheet_name="s3_ios")
    s3_android.to_excel(writer, sheet_name="s3_android")
    s4_ios.to_excel(writer, sheet_name="s4_ios")
    s4_android.to_excel(writer, sheet_name="s4_android")
    s5_ios.to_excel(writer, sheet_name="s5_ios")
    s5_android.to_excel(writer, sheet_name="s5_android")